This notebook runs the model on the preprocessed data. The goal is to predict if the patient will survive to its stay.

In [ ]:
import sys
sys.path.append('../')
import pandas as pd

# Reload data

In [ ]:
labs = pd.read_csv('data/labs_1_day.csv', index_col = [0, 1], header = [0, 1])
outcomes = pd.read_csv('data/outcomes_1_day.csv', index_col = 0)

In [ ]:
outcomes['Death'] = outcomes['Death'] < 8

In [ ]:
groups = outcomes[['ETHNICITY', 'GENDER', 'INSURANCE']]
groups.ETHNICITY = outcomes.ETHNICITY.str.contains('BLACK')
groups.GENDER = (outcomes.ETHNICITY == 'M')
groups.INSURANCE = (outcomes.INSURANCE == 'Private')

# Split 

In [ ]:
# Results path
results = 'results/classification' 

In [ ]:
training = pd.Series(outcomes.index.isin(outcomes.sample(frac = 0.8, random_state = 0).index), index = outcomes.index)

In [ ]:
print('Total patients: {}'.format(len(training)))
print('Training patients: {}'.format(training.sum()))

# Imputation

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import numpy as np

def imputation(train_index, data, groups, strategy = 'Median', add_missingness = False, max_iter = 10):
    imputed = data.add_suffix('_data').groupby('Patient').ffill().groupby('Patient').last() # Latest test

    if add_missingness:
        imputed = imputed.join(imputed.isna().add_suffix('_missing'))

    # Data to use to learn imputation
    train_data = imputed.loc[imputed.index.get_level_values('Patient').isin(train_index)]
    train_index = train_data.index
    
    # Compute fill value
    impute = train_data.mean()
    if strategy == 'Mean':
        imputed = imputed.transform(lambda x: x.fillna(impute[x.name]))

    if strategy == 'Group Mean':
        mean_group = train_data.groupby(groups.loc[train_index]).mean()
        imputed = imputed.groupby(groups).transform(lambda x: x.fillna(mean_group.loc[groups.loc[x.index[0]]][x.name]))
        imputed = imputed.transform(lambda x: x.fillna(impute[x.name])) # Replace any missing data

    if 'MICE' in strategy:
        if 'Group' in strategy:
            # Add group befoer splitting only for imputation
            imputed = imputed.join(groups)
            train_data = imputed.loc[train_index]
        # MICE Algorithm
        ## 1. Init with median imputation
        missing = imputed.isna()
        imputed = pd.DataFrame(SimpleImputer(strategy = "mean").fit(train_data.values).transform(imputed.values), index = imputed.index, columns = imputed.columns)

        ## 2. Iterate through columns
        ### Find columns with random values (start with the one with least)
        to_impute = missing.sum().sort_values()
        to_impute = to_impute[to_impute > 0]

        ### Impute one by one with regression until convergence
        for _ in range(max_iter):
            for c in to_impute.index:
                #### Take train points for which c is observed to train model
                train_data = imputed.loc[train_index][~missing.loc[train_index][c]]

                #### Fit regression
                lr = LinearRegression().fit(train_data.loc[:, imputed.columns != c].values, train_data[c].values)
                residuals = np.abs(lr.predict(train_data.loc[:, imputed.columns != c].values) - train_data[c])

                #### Draw with normal error
                prev = imputed.copy()
                imputed[c][missing[c]] = lr.predict(imputed.loc[:, imputed.columns != c][missing[c]].values) + np.random.normal(scale = np.std(residuals), size = missing[c].sum())
        if 'Group' in strategy:
            # Remove the group columns of imputed data
            imputed = imputed.iloc[:, :-1]

    return imputed

In [ ]:
from utils import Experiment

In [ ]:
hyperparams = {
    'penalty': ['l2'],
    'C': [0.01, 0.1, 1., 10],
    'solver': ['sag'], 
    'max_iter': [1000],
    'n_jobs': [-1]
}

In [ ]:
imputations = {
                'Mean': {'strategy': 'Mean'},
                'Mean Missing': {'strategy': 'Mean', 'add_missingness': True},

                'Group Ethnicity Mean': {'strategy': 'Group Mean', 'group': groups.ETHNICITY},
                'Group Sex Mean': {'strategy': 'Group Mean', 'group': groups.GENDER},
                'Group Insurance Mean': {'strategy': 'Group Mean', 'group': groups.INSURANCE},

                'Group Ethnicity Mean Missing': {'strategy': 'Group Mean', 'group': groups.ETHNICITY, 'add_missingness': True},
                'Group Sex Mean Missing': {'strategy': 'Group Mean', 'group': groups.GENDER, 'add_missingness': True},
                'Group Insurance Mean Missing': {'strategy': 'Group Mean', 'group': groups.INSURANCE, 'add_missingness': True},

                'MICE': {'strategy': 'MICE', 'n_iter': 10},
                'MICE Missing': {'strategy': 'MICE', 'n_iter': 10, 'add_missingness': True},

                'Group Ethnicity MICE': {'strategy': 'Group MICE', 'n_iter': 10, 'group': groups.ETHNICITY},
                'Group Sex MICE': {'strategy': 'Group MICE', 'n_iter': 10, 'group': groups.GENDER},
                'Group Insurance MICE': {'strategy': 'Group MICE', 'n_iter': 10, 'group': groups.INSURANCE},

                'Group Ethnicity MICE Missing': {'strategy': 'Group MICE', 'n_iter': 10, 'group': groups.ETHNICITY, 'add_missingness': True},
                'Group Sex MICE Missing': {'strategy': 'Group MICE', 'n_iter': 10, 'group': groups.GENDER, 'add_missingness': True},
                'Group Insurance MICE Missing': {'strategy': 'Group MICE', 'n_iter': 10, 'group': groups.INSURANCE, 'add_missingness': True},
              }

In [ ]:
for name, params in imputations.items():
    print('Imputation strategy: ', name)
    n_iter = params.pop('n_iter', 1)
    subgroup = params.pop('group', groups)

    predictions = []
    for iter in range(n_iter):
        last = imputation(training[training].index, labs, subgroup, **params)
        assert (last == -1).sum().sum() == 0, "Non imputed values"

        se = Experiment.create(model = 'log', hyper_grid = hyperparams, save = False, path = results + name)
        pred = se.train(last, outcomes.Death, training)
        if pred is None: break # Reload previous copy
        predictions.append(pred)
    else:
        # Average Multiple imputations models
        used = [p.Use for p in predictions][-1]
        predictions = pd.concat([p[1] for p in predictions], axis = 1)
        predictions = pd.concat({'Mean': predictions.mean(1), 'Std': predictions.std(1)}, axis = 1)
        se = Experiment.create(model = 'log', hyper_grid = hyperparams, path = results + name)
        se.save_results(predictions, used)